<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesi%C3%B3n_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilo-vega-169084b1/

---

## Guía: Fine-tuning y RAG (Retrieval-Augmented Generation)

Este notebook presenta **3 implementaciones prácticas**:

1. **Fine-tuning Básico** - Análisis de sentimientos con tweets
2. **RAG Simple** - Búsqueda básica en documentos
3. **RAG Base de Datos** - Sistema especializado con embeddings

### Requisitos
- **GPU:** Tesla T4 mínimo (Colab gratuito)
- **APIs:** Hugging Face token
- **Datos:** Tweets y dataset de ventas

## Configuración APIs
- **Hugging Face:**
  1. [Crear token](https://huggingface.co/settings/tokens)
  2. En Colab: 🔑 Secrets → Agregar `HF_TOKEN` → Pegar tu token

Cada sección es **independiente** y puede ejecutarse por separado.


In [ ]:
# 1. FINE-TUNING BÁSICO - ANÁLISIS DE SENTIMIENTOS

"""
Fine-tuning de un modelo pre-entrenado para análisis de sentimientos
usando el dataset de reviews de Amazon en español
"""

#0 Instalaciones
!pip install torch transformers datasets scikit-learn matplotlib -q

#1 Imports
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from sklearn.metrics import accuracy_score
import random
from collections import defaultdict

#2 Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")

#3 Cargar dataset de Amazon Reviews
print("Cargando dataset de Amazon Reviews...")
dataset = load_dataset("mteb/amazon_reviews_multi", "es", split="train")
print(f"Total ejemplos: {len(dataset)}")

#4 Balancear dataset por clases
print("\n📊 Analizando distribución de clases...")
class_counts = defaultdict(int)
for sample in dataset:
    class_counts[sample['label']] += 1

print("Distribución original:")
for label, count in sorted(class_counts.items()):
    print(f"  Clase {label}: {count:,} ejemplos")

#5 Crear dataset balanceado
samples_per_class = 500  # Muestras por clase para entrenamiento rápido
balanced_samples = []

# Organizar por clase
class_samples = defaultdict(list)
for sample in dataset:
    class_samples[sample['label']].append(sample)

# Balancear clases
random.seed(42)
for label, samples in class_samples.items():
    if len(samples) >= samples_per_class:
        selected = random.sample(samples, samples_per_class)
        balanced_samples.extend(selected)
    else:
        print(f"⚠️ Clase {label}: solo {len(samples)} ejemplos disponibles")
        balanced_samples.extend(samples)

print(f"\n✅ Dataset balanceado: {len(balanced_samples)} ejemplos")

#6 Extraer textos y etiquetas
texts = [sample['text'] for sample in balanced_samples]
labels = [sample['label'] for sample in balanced_samples]

# Verificar distribución final
final_distribution = defaultdict(int)
for label in labels:
    final_distribution[label] += 1

print("Distribución balanceada:")
for label, count in sorted(final_distribution.items()):
    print(f"  Clase {label}: {count} ejemplos")

#7 Mostrar ejemplos
print("\n📝 Ejemplos del dataset:")
for i in range(3):
    print(f"{i+1}. Texto: '{texts[i][:80]}...'")
    print(f"   Etiqueta: {labels[i]} (1=muy negativa, 5=muy positiva)")
    print()

#8 Cargar modelo y tokenizador
model_name = "bert-base-multilingual-cased"
print(f"Cargando modelo: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = len(set(labels))  # Número único de etiquetas
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
).to(device)

print(f"✅ Modelo cargado con {num_labels} clases")

#9 Tokenizar textos
print("Tokenizando textos...")
def tokenize_function(examples):
    return tokenizer(
        examples,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Tokenizar todos los textos
tokenized_inputs = tokenize_function(texts)

#10 Crear dataset de PyTorch
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # Ajustar a 0-indexado
        return item

    def __len__(self):
        return len(self.labels)

# Crear datasets
train_dataset = ReviewsDataset(tokenized_inputs, labels)

# Split para validación
split_idx = int(0.8 * len(train_dataset))
train_subset = torch.utils.data.Subset(train_dataset, range(split_idx))
eval_subset = torch.utils.data.Subset(train_dataset, range(split_idx, len(train_dataset)))

print(f"Train: {len(train_subset)}, Eval: {len(eval_subset)}")

#11 Configurar hiperparámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./amazon_model",

    # HIPERPARÁMETROS PRINCIPALES
    num_train_epochs=3,                    # Épocas - cuántas veces ve los datos
    per_device_train_batch_size=16,        # Tamaño batch - memoria GPU vs velocidad
    learning_rate=5e-5,                    # Tasa aprendizaje - velocidad de actualización
    weight_decay=0.01,                     # Regularización - previene overfitting

    # OPTIMIZACIÓN
    warmup_steps=200,                      # Pasos calentamiento - estabiliza inicio
    gradient_accumulation_steps=2,         # Acumular gradientes - simula batch más grande

    # EVALUACIÓN
    eval_strategy="epoch",                 # Evaluar cada época
    save_strategy="epoch",                 # Guardar cada época
    load_best_model_at_end=True,          # Cargar mejor modelo

    # LOGGING
    logging_dir="./logs",
    logging_steps=50,
    report_to=None,                       # Sin wandb

    # LÍMITES
    max_steps=150,                        # Máximo pasos para demo rápida
    save_total_limit=1,                   # Solo guardar mejor modelo
)

#12 Función de métricas
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

#13 Crear trainer con registro de pérdida
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.training_losses = []

    def log(self, logs):
        if "loss" in logs:
            self.training_losses.append(logs["loss"])
        super().log(logs)

#14 Configurar entrenador
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
)

#15 Entrenar modelo
print("\n🚀 Iniciando fine-tuning...")
trainer.train()
print("✅ Entrenamiento completado")

#16 Evaluar modelo final
print("\n📊 Evaluación final:")
results = trainer.evaluate()
for key, value in results.items():
    if key.startswith('eval_'):
        print(f"  {key}: {value:.4f}")

#17 Guardar modelo
model_path = "./amazon_sentiment_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"💾 Modelo guardado en: {model_path}")

#18 Visualizar pérdida de entrenamiento
if trainer.training_losses:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    plt.plot(trainer.training_losses)
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Pasos')
    plt.ylabel('Pérdida')
    plt.grid(True)
    plt.show()
    print(f"Pérdida inicial: {trainer.training_losses[0]:.4f}")
    print(f"Pérdida final: {trainer.training_losses[-1]:.4f}")

#19 Crear pipeline de inferencia
print("\n🤖 Creando pipeline de inferencia...")
sentiment_pipeline = pipeline(
    "text-classification",
    model=model_path,
    tokenizer=model_path,
    device=0 if torch.cuda.is_available() else -1
)

#20 Función de interpretación
def interpretar_sentimiento(resultado):
    """Convierte resultado numérico a texto legible"""
    # Pipeline devuelve LABEL_0, LABEL_1, etc.
    if resultado['label'].startswith('LABEL_'):
        label_num = int(resultado['label'].split('_')[1])

        # Mapear a estrellas (0-indexado a 1-5 estrellas)
        estrellas = label_num + 1

        if estrellas == 1:
            sentimiento = "⭐ Muy Negativo"
        elif estrellas == 2:
            sentimiento = "⭐⭐ Negativo"
        elif estrellas == 3:
            sentimiento = "⭐⭐⭐ Neutral"
        elif estrellas == 4:
            sentimiento = "⭐⭐⭐⭐ Positivo"
        elif estrellas == 5:
            sentimiento = "⭐⭐⭐⭐⭐ Muy Positivo"
        else:
            sentimiento = f"Desconocido ({estrellas} estrellas)"
    else:
        sentimiento = resultado['label']

    return {
        'sentimiento': sentimiento,
        'confianza': resultado['score']
    }

#21 Probar modelo con ejemplos
test_reviews = [
    "Me encanta este producto, es excelente y funciona perfectamente",
    "Terrible compra, no funciona y es de muy mala calidad",
    "El producto está bien, nada espectacular pero cumple su función",
    "¡Increíble! Superó todas mis expectativas, lo recomiendo totalmente",
    "No me gustó para nada, perdí mi dinero con esta compra",
    "Producto decente por el precio que tiene"
]

print("\n🧪 PROBANDO MODELO:")
print("=" * 50)

for i, review in enumerate(test_reviews, 1):
    resultado = sentiment_pipeline(review)[0]
    interpretado = interpretar_sentimiento(resultado)

    print(f"{i}. Review: \"{review[:60]}{'...' if len(review) > 60 else ''}\"")
    print(f"   Sentimiento: {interpretado['sentimiento']}")
    print(f"   Confianza: {interpretado['confianza']:.3f}")
    print()

#22 Función de inferencia personalizada (alternativa)
def analizar_review(texto):
    """Función simple para analizar sentimiento de una review"""
    resultado = sentiment_pipeline(texto)[0]
    interpretado = interpretar_sentimiento(resultado)

    print(f"📝 Review: \"{texto}\"")
    print(f"🎯 Sentimiento: {interpretado['sentimiento']}")
    print(f"📊 Confianza: {interpretado['confianza']:.1%}")

    return interpretado

print("\n🔍 Ejemplo de uso de función personalizada:")
analizar_review("Este dispositivo es fantástico, me encanta usarlo todos los días")

print(f"\n✅ FINE-TUNING COMPLETADO EXITOSAMENTE!")
print(f"📁 Modelo guardado en: {model_path}")
print(f"🎯 Accuracy final: {results.get('eval_accuracy', 'N/A'):.4f}")
print(f"⚡ Listo para usar en producción!")

In [ ]:
# RAG SIMPLE CON GROQ API
# Sistema de búsqueda en documentos usando TF-IDF y Groq para generación

# Instalación de dependencias
!pip install groq scikit-learn PyPDF2 python-docx pandas openpyxl -q

import os
import pandas as pd
import numpy as np
from groq import Groq
from google.colab import userdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader
from docx import Document

# Configuración del sistema
GROQ_API_KEY = userdata.get('GROQ_KEY')
client = Groq(api_key=GROQ_API_KEY)
folder_path = '/content/carpeta_rag'

# Variables globales del sistema
vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
chunks = []
vectors = None

def load_documents(folder_path):
    """Carga y procesa documentos de múltiples formatos"""
    text = ''
    os.makedirs(folder_path, exist_ok=True)

    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)

        try:
            if filename.endswith('.txt'):
                with open(filepath, 'r', encoding='utf-8') as f:
                    text += f.read() + '\n'

            elif filename.endswith('.pdf'):
                reader = PdfReader(filepath)
                for page in reader.pages:
                    text += page.extract_text() + '\n'

            elif filename.endswith('.docx'):
                doc = Document(filepath)
                for paragraph in doc.paragraphs:
                    text += paragraph.text + '\n'

            elif filename.endswith('.csv'):
                df = pd.read_csv(filepath)
                text += df.to_string() + '\n'

            elif filename.endswith('.xlsx'):
                df = pd.read_excel(filepath)
                text += df.to_string() + '\n'
        except:
            continue

    return text

def create_chunks(text, chunk_size=500):
    """Divide el texto en fragmentos de tamaño manejable"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.strip()) > 100:
            chunks.append(chunk.strip())

    return chunks

def search_relevant_chunks(query, top_k=3):
    """Encuentra fragmentos más relevantes usando similitud coseno"""
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, vectors)[0]

    # Obtener índices ordenados por relevancia
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    relevant_chunks = []
    for idx in top_indices:
        if similarities[idx] > 0.1:  # Umbral mínimo de relevancia
            relevant_chunks.append(chunks[idx])

    return relevant_chunks

def generate_answer(query, context):
    """Genera respuesta usando Groq API con el contexto encontrado"""
    if not context:
        return "No encontré información relevante en los documentos."

    prompt = f"""Basándote en esta información:

{context}

Pregunta: {query}

Instrucciones:
- Responde solo con información del contexto
- Si no hay información suficiente, di que no puedes responder
- Responde en español de forma clara y concisa"""

    try:
        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=400,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generando respuesta: {e}"

def inicializar_rag():
    """Inicializa el sistema RAG cargando documentos y creando índices"""
    global chunks, vectors, vectorizer

    print("Cargando documentos...")
    documents = load_documents(folder_path)

    if not documents.strip():
        print("No se encontraron documentos en la carpeta.")
        return False

    print("Creando fragmentos de texto...")
    chunks = create_chunks(documents)

    print("Generando índice de búsqueda...")
    vectors = vectorizer.fit_transform(chunks)

    print(f"Sistema listo: {len(chunks)} fragmentos procesados.")
    return True

# Inicialización del sistema
inicializar_rag()

# BLOQUE DE INFERENCIA - Ejecutar por separado
query = "¿De qué tratan estos documentos?"  # Cambia tu pregunta aquí
context = '\n\n'.join(search_relevant_chunks(query))
respuesta = client.chat.completions.create(model="llama3-8b-8192", messages=[{"role": "user", "content": f"Contexto: {context}\nPregunta: {query}\nResponde solo con información del contexto en español."}], max_tokens=400).choices[0].message.content
print(query)
print(respuesta)

In [ ]:
# RAG BASE DE DATOS CON GROQ API


# ===== INSTALACIÓN =====
!pip install groq sentence-transformers pandas numpy scikit-learn -q

# ===== CÓDIGO RAG MINIMALISTA =====
import pandas as pd
import numpy as np
import sqlite3
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq
from google.colab import userdata

class RAGSimple:
    def __init__(self):
        print("Iniciando RAG...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.conn = None
        self.table_name = 'data'
        self.column_embeddings = {}
        self.column_types = {}

    def cargar_datos(self, csv_url):
        """Cargar CSV y crear base de datos"""
        print("Cargando datos...")
        df = pd.read_csv(csv_url)
        self.conn = sqlite3.connect(':memory:')
        df.to_sql(self.table_name, self.conn, index=False, if_exists='replace')

        # Analizar columnas
        for col in df.columns:
            if df[col].dtype in ['int64', 'float64']:
                self.column_types[col] = 'numeric'
                desc = f"{col} valores numéricos entre {df[col].min()} y {df[col].max()}"
            elif df[col].nunique() <= 20:
                self.column_types[col] = 'categorical'
                valores = ', '.join(map(str, df[col].unique()[:5]))
                desc = f"{col} categorías como: {valores}"
            else:
                self.column_types[col] = 'text'
                desc = f"{col} texto libre"

            self.column_embeddings[col] = self.model.encode([desc])[0]

        print(f"✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
        return df.shape

    def encontrar_columnas(self, pregunta):
        """Encontrar columnas relevantes"""
        pregunta_emb = self.model.encode([pregunta])[0]
        scores = []

        for col, col_emb in self.column_embeddings.items():
            sim = cosine_similarity([pregunta_emb], [col_emb])[0][0]
            scores.append((col, sim))

        return sorted(scores, key=lambda x: x[1], reverse=True)[:3]

    def generar_sql(self, pregunta):
        """Generar SQL inteligente"""
        cols_relevantes = [col for col, _ in self.encontrar_columnas(pregunta)]
        cols_numericas = [c for c in cols_relevantes if self.column_types.get(c) == 'numeric']
        cols_categoricas = [c for c in cols_relevantes if self.column_types.get(c) == 'categorical']

        pregunta_lower = pregunta.lower()

        # Detección de intención mejorada
        if any(palabra in pregunta_lower for palabra in ['más vendido', 'más popular', 'qué producto', 'cuáles productos']):
            # Productos más vendidos por cantidad
            if cols_numericas and cols_categoricas:
                return f"""
                SELECT {cols_categoricas[0]} as producto,
                       SUM({cols_numericas[0]}) as total_cantidad,
                       COUNT(*) as num_ventas
                FROM {self.table_name}
                GROUP BY {cols_categoricas[0]}
                ORDER BY total_cantidad DESC
                LIMIT 10
                """

        if any(palabra in pregunta_lower for palabra in ['método pago', 'forma pago', 'cómo pagan', 'pago más común']):
            # Análisis de métodos de pago
            return f"""
            SELECT Método_pago,
                   COUNT(*) as cantidad,
                   ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM {self.table_name}), 1) as porcentaje
            FROM {self.table_name}
            GROUP BY Método_pago
            ORDER BY cantidad DESC
            """

        if any(palabra in pregunta_lower for palabra in ['ingresos', 'revenue', 'dinero', 'ganancias']):
            # Análisis de ingresos
            if 'Precio_unitario' in self.column_types and 'Cantidad' in self.column_types:
                return f"""
                SELECT {cols_categoricas[0] if cols_categoricas else 'Producto'} as categoria,
                       SUM(Precio_unitario * Cantidad) as total_ingresos,
                       AVG(Precio_unitario * Cantidad) as ingreso_promedio
                FROM {self.table_name}
                GROUP BY {cols_categoricas[0] if cols_categoricas else 'Producto'}
                ORDER BY total_ingresos DESC
                LIMIT 10
                """

        if any(palabra in pregunta_lower for palabra in ['ciudad', 'sucursal', 'dónde', 'ubicación']):
            # Análisis por ubicación
            return f"""
            SELECT Sucursal,
                   COUNT(*) as num_ventas,
                   SUM(Precio_unitario * Cantidad) as total_ingresos
            FROM {self.table_name}
            GROUP BY Sucursal
            ORDER BY num_ventas DESC
            """

        if any(palabra in pregunta_lower for palabra in ['promedio', 'precio promedio', 'categoría']):
            # Análisis de precios por categoría
            return f"""
            SELECT Categoría,
                   COUNT(*) as num_productos,
                   AVG(Precio_unitario) as precio_promedio,
                   MIN(Precio_unitario) as precio_min,
                   MAX(Precio_unitario) as precio_max
            FROM {self.table_name}
            GROUP BY Categoría
            ORDER BY precio_promedio DESC
            """

        # Query general mejorada
        select_cols = ', '.join(cols_relevantes[:3]) if cols_relevantes else '*'
        return f"SELECT {select_cols} FROM {self.table_name} LIMIT 15"

    def ejecutar_consulta(self, sql):
        """Ejecutar SQL"""
        try:
            sql_limpio = ' '.join(line.strip() for line in sql.strip().split('\n') if line.strip())
            resultado = pd.read_sql_query(sql_limpio, self.conn)
            return sql_limpio, resultado
        except Exception as e:
            print(f"Error SQL: {e}")
            sql_simple = f"SELECT * FROM {self.table_name} LIMIT 10"
            resultado = pd.read_sql_query(sql_simple, self.conn)
            return sql_simple, resultado

    def responder(self, pregunta, usar_groq=True):
        """Responder pregunta completa"""
        try:
            # Generar y ejecutar SQL
            sql = self.generar_sql(pregunta)
            sql_ejecutado, datos = self.ejecutar_consulta(sql)

            print(f"SQL ejecutado: {sql_ejecutado}")
            print(f"Resultados: {len(datos)} filas\n")

            # Crear contexto
            contexto = f"""Pregunta: {pregunta}
SQL ejecutado: {sql_ejecutado}
Resultados: {len(datos)} filas encontradas

"""

            if not datos.empty:
                # Estadísticas automáticas
                cols_numericas = datos.select_dtypes(include=[np.number]).columns
                if len(cols_numericas) > 0:
                    contexto += "Estadísticas calculadas:\n"
                    for col in cols_numericas[:3]:
                        contexto += f"- {col}: total={datos[col].sum():.2f}, promedio={datos[col].mean():.2f}, máximo={datos[col].max():.2f}\n"
                    contexto += "\n"

                contexto += "Datos encontrados:\n"
                contexto += datos.to_string(index=False)

            # Respuesta con Groq si está disponible
            if usar_groq:
                try:
                    groq_key = userdata.get('GROQ_KEY')
                    client = Groq(api_key=groq_key)

                    response = client.chat.completions.create(
                        model="llama3-8b-8192",
                        messages=[{
                            "role": "user",
                            "content": f"""Analiza estos datos y responde de forma clara y específica:

{contexto}

Instrucciones:
- Responde basándote ÚNICAMENTE en los datos mostrados
- Da números exactos y estadísticas precisas
- Responde en español de forma profesional
- Si hay un ranking, muestra los top 3-5 elementos

Pregunta: {pregunta}"""
                        }],
                        max_tokens=400
                    )

                    return response.choices[0].message.content

                except Exception as e:
                    print(f"Error con Groq: {e}")
                    usar_groq = False

            # Respuesta automática si no hay Groq
            if not datos.empty:
                respuesta = f"Análisis para: {pregunta}\n\n"
                respuesta += f"Encontré {len(datos)} resultados:\n\n"
                respuesta += datos.head(10).to_string(index=False)

                if len(cols_numericas) > 0:
                    respuesta += f"\n\nEstadísticas principales:\n"
                    for col in cols_numericas[:2]:
                        respuesta += f"- {col}: Total = {datos[col].sum():.2f}, Promedio = {datos[col].mean():.2f}\n"

                return respuesta
            else:
                return "No se encontraron datos para esta consulta."

        except Exception as e:
            return f"Error procesando consulta: {e}"

# ===== FUNCIÓN DE USO SIMPLE =====
def iniciar_rag():
    """Inicializar RAG con el dataset"""
    rag = RAGSimple()
    rag.cargar_datos('https://raw.githubusercontent.com/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/main/datos_tienda_ropa.csv')
    return rag

def preguntar(rag, pregunta):
    """Hacer una pregunta al RAG"""
    return rag.responder(pregunta, usar_groq=True)

# ===== CÓDIGO DE PRUEBA =====
if __name__ == "__main__":
    print("🚀 INICIANDO SISTEMA RAG")
    rag = iniciar_rag()

    # Preguntas de prueba
    preguntas_test = [
        "¿cuáles son los productos más vendidos?",
        "¿cuál es el método de pago más común?",
        "¿qué producto genera más ingresos?",
        "¿cuál es el precio promedio por categoría?"
    ]

    for i, pregunta in enumerate(preguntas_test, 1):
        print(f"\n{'='*60}")
        print(f"PREGUNTA {i}: {pregunta}")
        print('='*60)

        respuesta = preguntar(rag, pregunta)
        print(respuesta)